<a href="https://colab.research.google.com/github/tubagokhan/ADGM/blob/main/TextClassificationWithoutDistinction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install transformers[torch]
!pip install accelerate -U
!pip install tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 59.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 35.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 90.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 35.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 5.5 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import json
import torch
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm

# Load your JSON data
with open('/content/drive/Othercomputers/MBZUAI/MBZUAI/Codes/COBS_All_Labels_Phrases.json', 'r', encoding='utf-8-sig') as file:
    data = json.load(file)

# Extract labels and phrases
labels = [item["label_"] for item in data]
phrases = [item["phrase"] for item in data]

# Split the data into training, validation, and testing sets
train_texts, test_texts, train_labels, test_labels = train_test_split(
    phrases, labels, test_size=0.2, random_state=42
)

# Further split the training data into training and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_texts, train_labels, test_size=0.1, random_state=42
)

# Load BERT tokenizer and encode the data
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
train_encodings = tokenizer(train_texts, padding=True, truncation=True, return_tensors="pt")
val_encodings = tokenizer(val_texts, padding=True, truncation=True, return_tensors="pt")
test_encodings = tokenizer(test_texts, padding=True, truncation=True, return_tensors="pt")

# Encode labels using LabelEncoder
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
val_labels_encoded = label_encoder.transform(val_labels)
test_labels_encoded = label_encoder.transform(test_labels)

unique_labels = set(labels)
num_classes = len(unique_labels)
print(f"Number of unique classes: {num_classes}")

# Create BERT-based text classification model
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=num_classes)

# Move the model and data to the GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
train_encodings = {key: value.to(device) for key, value in train_encodings.items()}
val_encodings = {key: value.to(device) for key, value in val_encodings.items()}
test_encodings = {key: value.to(device) for key, value in test_encodings.items()}
train_labels_encoded = torch.tensor(train_labels_encoded, device=device)
val_labels_encoded = torch.tensor(val_labels_encoded, device=device)
test_labels_encoded = torch.tensor(test_labels_encoded, device=device)

# Fine-tuning hyperparameters
batch_size = 64
max_seq_length = 128
learning_rate = 2e-5
num_epochs = 50
gradient_accumulation_steps = 4

# Create data loaders
train_dataset = TensorDataset(train_encodings["input_ids"], train_encodings["attention_mask"], train_labels_encoded)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Optimization and loss function
optimizer = AdamW(model.parameters(), lr=learning_rate)
loss_fn = torch.nn.CrossEntropyLoss()

# Early stopping parameters
patience = 3
best_val_accuracy = 0
no_improvement_count = 0

# Training loop
model.train()
for epoch in range(num_epochs):
    total_loss = 0
    tqdm_data = tqdm(enumerate(train_loader), total=len(train_loader))
    for step, batch in tqdm_data:
        input_ids, attention_mask, labels = batch
        inputs = {"input_ids": input_ids, "attention_mask": attention_mask, "labels": labels}
        outputs = model(**inputs)
        loss = outputs.loss

        loss.backward()

        if (step + 1) % gradient_accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()

        total_loss += loss.item()
        avg_loss = total_loss / (step + 1)

        tqdm_data.set_description(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}")
    print(f"Epoch {epoch + 1} - Average Loss: {total_loss / len(train_loader)}")

    # Evaluation on validation data after each epoch
    model.eval()
    with torch.no_grad():
        val_outputs = model(**val_encodings)
        val_predicted_labels = torch.argmax(val_outputs.logits, dim=1)
        val_accuracy = accuracy_score(val_labels_encoded.cpu().numpy(), val_predicted_labels.cpu().numpy())
        print(f"Validation Accuracy (Epoch {epoch+1}): {val_accuracy:.4f}")
        f1 = f1_score(val_labels_encoded.cpu().numpy(), val_predicted_labels.cpu().numpy(), average='micro')
        print(f"Epoch {epoch + 1} - F1 Score: {f1}")

        # Implement early stopping
        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            no_improvement_count = 0

            # Save the model checkpoint with the best validation accuracy
            model_checkpoint_path = '/content/drive/Othercomputers/MBZUAI/MBZUAI/Codes/best_model.pt'
            torch.save(model.state_dict(), model_checkpoint_path)
            print("Saved model checkpoint with validation accuracy: {:.4f}".format(val_accuracy))
        else:
            no_improvement_count += 1
            if no_improvement_count >= patience:
                print("Early stopping triggered. No improvement for {} epochs.".format(patience))
                break  # Stop training


Number of unique classes: 9


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Epoch 1/50, Loss: 1.6640: 100%|██████████| 209/209 [00:57<00:00,  3.61it/s]


Epoch 1 - Average Loss: 1.6640228557814822
Validation Accuracy (Epoch 1): 0.5618
Epoch 1 - F1 Score: 0.5617825793382849
Saved model checkpoint with validation accuracy: 0.5618


Epoch 2/50, Loss: 0.9849: 100%|██████████| 209/209 [00:54<00:00,  3.86it/s]


Epoch 2 - Average Loss: 0.9848770319559927
Validation Accuracy (Epoch 2): 0.6428
Epoch 2 - F1 Score: 0.6428089128966914
Saved model checkpoint with validation accuracy: 0.6428


...

Epoch 3 - Average Loss: 0.7691132352112583
Validation Accuracy (Epoch 3): 0.6354
Epoch 3 - F1 Score: 0.6353814989871708


Epoch 4/50, Loss: 0.6474: 100%|██████████| 209/209 [00:54<00:00,  3.86it/s]


Epoch 4 - Average Loss: 0.6474336117648622
Validation Accuracy (Epoch 4): 0.6293
Epoch 4 - F1 Score: 0.6293045239702904


Epoch 5/50, Loss: 0.5740: 100%|██████████| 209/209 [00:54<00:00,  3.86it/s]


Epoch 5 - Average Loss: 0.5739510820813156
Validation Accuracy (Epoch 5): 0.6428
Epoch 5 - F1 Score: 0.6428089128966914
Early stopping triggered. No improvement for 3 epochs.


In [10]:
# Load the saved model for evaluation on the GPU
loaded_model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=num_classes)
loaded_model.load_state_dict(torch.load(model_checkpoint_path))
loaded_model.to(device)  # Move the loaded model to the GPU
loaded_model.eval()  # Set the loaded model in evaluation mode for inference

# Evaluation on test data
from sklearn.metrics import classification_report, f1_score

with torch.no_grad():
    test_outputs = loaded_model(**test_encodings)  # Use loaded_model for inference
    test_predicted_labels = torch.argmax(test_outputs.logits, dim=1)
    test_accuracy = accuracy_score(test_labels_encoded.cpu().numpy(), test_predicted_labels.cpu().numpy())
    print(f"Test Accuracy: {test_accuracy:.4f}")
    f1 = f1_score(test_labels_encoded.cpu().numpy(), test_predicted_labels.cpu().numpy(), average='micro')
    print("Micro F1-Score:", f1)

    test_report = classification_report(test_labels_encoded.cpu().numpy(), test_predicted_labels.cpu().numpy())
    print("Test Classification Report:\n", test_report)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [28]:
unique_labels


{'ACT', 'DEF', 'ENT', 'FS', 'MIT', 'PERM', 'PROD', 'RISK', 'TECH'}

In [45]:
import json
import torch
from transformers import BertTokenizer, BertForSequenceClassification
import numpy as np

# Load the new JSON file containing phrases
with open('/content/drive/Othercomputers/MBZUAI/MBZUAI/Codes/samplePhrasesandTags100.json', 'r', encoding='utf-8-sig') as json_file:
    data = json.load(json_file)

# Extract phrases and original labels from the new data
phrases = [item["phrase"] for item in data]
original_labels = [item["label_"] for item in data]

# Preprocess the new data
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
encoded_phrases = tokenizer(phrases, padding=True, truncation=True, return_tensors="pt")

# Move the new data to the GPU
encoded_phrases = {key: value.to(device) for key, value in encoded_phrases.items()}

# Use the pre-trained model to predict labels
with torch.no_grad():
    model.eval()
    outputs = model(**encoded_phrases)
    predicted_labels = torch.argmax(outputs.logits, dim=1)

# Define your custom label encoder for the missing label 'PERM'
class CustomLabelEncoder:
    def transform(self, labels):
        return [label if label != 'PERM' else '8' for label in labels]

# Create an instance of your custom label encoder
label_encoder = CustomLabelEncoder()

# Decode the predicted labels
predicted_labels_decoded = label_encoder.transform(predicted_labels)

# Define a reverse label mapping
reverse_label_mapping = {
    '8': 'PERM',
    '0': 'ACT',  # Replace '0' with the corresponding label
    '1': 'DEF',
    '2': 'ENT',
    '3': 'FS',
    '4': 'MIT',
    '5': 'PROD',
    '6': 'RISK',
    '7': 'TECH',# Replace '1' with the corresponding label
    # Add mappings for other labels as needed
}

# Convert numeric labels (tensors) to text labels
predicted_labels_text = [reverse_label_mapping[str(label.item())] for label in predicted_labels]

# Print the original phrases and their predicted labels
for phrase, original_label, predicted_label, predicted_label_value in zip(phrases, original_labels, predicted_labels_text, predicted_labels.tolist()):
    print(f"Phrase: {phrase}")
    print(f"KG Label: {original_label}")
    print(f"Predicted Label: {predicted_label}")
    print()

Phrase: indefinite
KG Label: FS
Predicted Label: TECH

Phrase: Executive/CE
KG Label: MIT
Predicted Label: ENT

Phrase: Illustrative Regulatory Framework
KG Label: TECH
Predicted Label: MIT

Phrase: might take in the medium term (i.e. five to ten years
KG Label: MIT
Predicted Label: MIT

Phrase: We propose to include a section in the concept paper that describes a future
KG Label: MIT
Predicted Label: MIT

Phrase: an example of a DeFi insurance protocol
KG Label: RISK
Predicted Label: TECH

Phrase: except that this tends to focus on risks specific to DeFi such as the risk of a hack or of a failure in a smart contract
KG Label: MIT
Predicted Label: TECH

Phrase: insurance
KG Label: FS
Predicted Label: FS

Phrase: hack
KG Label: ACT
Predicted Label: PERM

Phrase: market
KG Label: FS
Predicted Label: FS

Phrase: regulators
KG Label: DEF
Predicted Label: ENT

Phrase: regime from the industry and other regulators to test
KG Label: MIT
Predicted Label: MIT

Phrase: We would invite comment
KG